In [1]:
symbols = ['BTCUSDT','ETHUSDT','BNBUSDT','SOLUSDT','ADAUSDT','TRXUSDT','AVAXUSDT','TONUSDT','SUIUSDT','HBARUSDT','BCHUSDT',
           'NEARUSDT','APTUSDT','ICPUSDT','VETUSDT','XMRUSDT','TAOUSDT','OMUSDT','FILUSDT','ALGOUSDT','FTMUSDT','TIAUSDT',
           'THETAUSDT','INJUSDT','SEIUSDT','GALAUSDT','XTZUSDT','KAIAUSDT','FLOWUSDT','DYDXUSDT','ARUSDT','IOTAUSDT','EGLDUSDT'
           ,'CHZUSDT','GNOUSDT','XECUSDT','VANAUSDT','ROSEUSDT','KAVAUSDT','ASTRUSDT','ENJUSDT','CELOUSDT','ONEUSDT','ELFUSDT',
           'KDAUSDT','POLYXUSDT','DCRUSDT','SKLUSDT','GLMRUSDT','VANRYUSDT','CHRUSDT','BBUSDT','SAGAUSDT','HIVEUSDT','WAXPUSDT',
           'ONGUSDT','MOVRUSDT','DUSKUSDT','SYSUSDT','OMNIUSDT','LTOUSDT','CTXCUSDT','REIUSDT']

In [10]:
import os
import pandas as pd
from functools import reduce

def get_data(path, symbols):
    df = pd.read_csv(path)
    df = df.drop(columns = df.columns[-1]).reset_index()
    df.drop(columns = df.columns[0], inplace = True)
    df.drop(index = 0, inplace = True)
    df.columns = ['date', 'coin', 'open', 'high', 'low', 'close', 'volume', 'volume_in_dollars']

    if not df['coin'].iloc[0] in symbols:
        return
    # Clean the date column by stripping whitespace
    df['date'] = df['date'].str.strip()
    # Parse the date column with mixed format
    df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
    
    df.set_index([df.columns[0], df.columns[1]], inplace = True)
    df = df.unstack()
    return df
    

def process_folder(folder_path, symbols):
    # Get all CSV files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    dfs = []
    
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = get_data(file_path, symbols)
        if df is not None:
            dfs.append(df)
    

    # Get the union of all indices (dates) to align the data
    all_dates = reduce(pd.Index.union, [df.index.get_level_values(0) for df in dfs])

    # Reindex all DataFrames to the same set of dates (adding NaNs where data is missing)
    dfs_aligned = [df.reindex(all_dates, level=0, fill_value=None) for df in dfs]

    # Concatenate all DataFrames
    concatenated_df = pd.concat(dfs_aligned, axis=1)
    concatenated_df = concatenated_df.sort_index(axis=1)
    concatenated_df = concatenated_df.apply(pd.to_numeric, errors='coerce', downcast='float')
    
    
    return concatenated_df

def upload_data_to_csv(df):
    # Upload the data to CSV file
    df.to_csv('all_data.csv')

# Example usage:
folder_path = r'C:\Users\yassi\OneDrive\Documents\Trading\Algo Trading Projects\Algo Business\data\Binance Data (CSV)'
result_df = process_folder(folder_path, symbols)
upload_data_to_csv(result_df)
result_df



C:\Users\yassi\AppData\Local\Temp\ipykernel_22248\3403350183.py:6: DtypeWarning: Columns (0,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


close                                           \
coin                ADAUSDT ALGOUSDT ARUSDT AVAXUSDT BCHUSDT BNBUSDT   
date                                                                   
2017-08-17 04:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 05:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 06:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 07:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 08:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
...                     ...      ...    ...      ...     ...     ...   
2024-12-27 19:00:00  0.8929   0.3395  16.98    37.19   447.1     NaN   
2024-12-27 20:00:00  0.8887   0.3374  16.97    36.99   445.0     NaN   
2024-12-27 21:00:00  0.8759   0.3324  16.73    36.63   440.8     NaN   
2024-12-27 22:00:00  0.8744   0.3325  16.73    36.45   440.9     NaN   
2024-12-27 23:00:00  0.8771   0.3331  16.74    36.56   441.3     NaN   

                                                        ... volume_in_dollars  \
coin                  BTCUSDT CELOUSDT CHRUSDT CHZUSDT  ...           SKLUSDT   
date                                                    ...                     
2017-08-17 04:00:00   4308.83      NaN     NaN     NaN  ...               NaN   
2017-08-17 05:00:00   4315.32      NaN     NaN     NaN  ...               NaN   
2017-08-17 06:00:00   4324.35      NaN     NaN     NaN  ...               NaN   
2017-08-17 07:00:00   4349.99      NaN     NaN     NaN  ...               NaN   
2017-08-17 08:00:00   4360.69      NaN     NaN     NaN  ...               NaN   
...                       ...      ...     ...     ...  ...               ...   
2024-12-27 19:00:00   94750.0   0.6782  0.2394     NaN  ...      147597.57314   
2024-12-27 20:00:00  94588.12   0.6756   0.238     NaN  ...       92097.98299   
2024-12-27 21:00:00  94526.07   0.6661  0.2353     NaN  ...       93698.29035   
2024-12-27 22:00:00  94437.75   0.6668  0.2349     NaN  ...       54746.25489   
2024-12-27 23:00:00  94299.03   0.6697  0.2359     NaN  ...      106334.05459   

                                                                          \
coin                        SOLUSDT     SYSUSDT THETAUSDT        TRXUSDT   
date                                                                       
2017-08-17 04:00:00             NaN         NaN       NaN            NaN   
2017-08-17 05:00:00             NaN         NaN       NaN            NaN   
2017-08-17 06:00:00             NaN         NaN       NaN            NaN   
2017-08-17 07:00:00             NaN         NaN       NaN            NaN   
2017-08-17 08:00:00             NaN         NaN       NaN            NaN   
...                             ...         ...       ...            ...   
2024-12-27 19:00:00   6679103.77139    62881.21       NaN  5721538.54161   
2024-12-27 20:00:00   8412081.50078  27205.6398       NaN  4796173.85633   
2024-12-27 21:00:00   6719660.39419  37773.5518       NaN  4429262.00457   
2024-12-27 22:00:00  10751316.32927  54086.3708       NaN  7739229.38011   
2024-12-27 23:00:00  14597166.41066  81340.7657       NaN  3885174.56787   

                                                                         \
coin                       VETUSDT     WAXPUSDT         XECUSDT XMRUSDT   
date                                                                      
2017-08-17 04:00:00            NaN          NaN             NaN     NaN   
2017-08-17 05:00:00            NaN          NaN             NaN     NaN   
2017-08-17 06:00:00            NaN          NaN             NaN     NaN   
2017-08-17 07:00:00            NaN          NaN             NaN     NaN   
2017-08-17 08:00:00            NaN          NaN             NaN     NaN   
...                            ...          ...             ...     ...   
2024-12-27 19:00:00  881844.023617  30357.62666  28784.19932494     NaN   
2024-12-27 20:00:00  524184.977356  50791.34977  32587.28112322

In [3]:
single_df_path = r"C:\Users\yassi\OneDrive\Documents\Trading\Algo Trading Projects\Algo Business\data\Binance Data (CSV)\Binance_BNBUSDT_1h.csv"
single_df = get_data(single_df_path, symbols)
single_df


,open,high,low,close,volume,volume_in_dollars
coin,BNBUSDT,BNBUSDT,BNBUSDT,BNBUSDT,BNBUSDT,BNBUSDT
date,,,,,,
2017-11-06 03:00:00,1.5,1.799,0.5,1.7,649.12,725.12142
2017-11-06 04:00:00,1.3,1.65,1.3,1.6479,8147.72,12708.527421
2017-11-06 05:00:00,1.5457,1.5525,1.5455,1.5458,6628.2,10265.341087
2017-11-06 06:00:00,1.5458,1.681,1.5387,1.681,22767.9,36507.144422
2017-11-06 07:00:00,1.6809,1.6809,1.6,1.625,14938.73,24278.726205
...,...,...,...,...,...,...
2024-12-26 19:00:00,692.97,693.33,688.5,692.22,5759.781,3978311.3611
2024-12-26 20:00:00,692.21,692.47,686.31,689.0,6824.042,4700692.80855


In [4]:
single_df.columns.levels[1]

Index(['BNBUSDT'], dtype='object', name='coin')

In [5]:
folder_path = r'C:\Users\yassi\OneDrive\Documents\Trading\Algo Trading Projects\Algo Business\data\Binance Data (CSV)'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

csv_files

['Binance_1INCHBUSD_1h.csv',
 'Binance_1INCHDOWNUSDT_1h.csv',
 'Binance_1INCHUPUSDT_1h.csv',
 'Binance_1INCHUSDT_1h.csv',
 'Binance_AAVEDOWNUSDT_1h.csv',
 'Binance_AAVEUPUSDT_1h.csv',
 'Binance_AAVEUSDT_1h.csv',
 'Binance_ACMUSDT_1h.csv',
 'Binance_ADADOWNUSDT_1h.csv',
 'Binance_ADAUPUSDT_1h.csv',
 'Binance_ADAUSDT_1h.csv',
 'Binance_AGLDUSDT_1h.csv',
 'Binance_AIONUSDT_1h.csv',
 'Binance_AKROUSDT_1h.csv',
 'Binance_ALGOUSDT_1h.csv',
 'Binance_ALICEUSDT_1h.csv',
 'Binance_ALPACAUSDT_1h.csv',
 'Binance_ALPHAUSDT_1h.csv',
 'Binance_ANKRUSDT_1h.csv',
 'Binance_ANTUSDT_1h.csv',
 'Binance_ARDRUSDT_1h.csv',
 'Binance_ARPAUSDT_1h.csv',
 'Binance_ARUSDT_1h.csv',
 'Binance_ASRUSDT_1h.csv',
 'Binance_ATAUSDT_1h.csv',
 'Binance_ATMUSDT_1h.csv',
 'Binance_ATOMUSDT_1h.csv',
 'Binance_AUDIOUSDT_1h.csv',
 'Binance_AUDUSDT_1h.csv',
 'Binance_AUTOUSDT_1h.csv',
 'Binance_AVAUSDT_1h.csv',
 'Binance_AVAXUSDT_1h.csv',
 'Binance_AXSUSDT_1h.csv',
 'Binance_BADGERUSDT_1h.csv',
 'Binance_BAKEUSDT_1h.csv',
 'Bi

In [16]:
dfs = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        df = get_data(file_path, symbols)
    except Exception as e:
        print(f"Error processing file {file}: {e}")
    if df is not None:
        dfs.append(df)

dfs

C:\Users\yassi\AppData\Local\Temp\ipykernel_22248\3403350183.py:6: DtypeWarning: Columns (0,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


[                        open     high      low    close      volume  \
 coin                 ADAUSDT  ADAUSDT  ADAUSDT  ADAUSDT     ADAUSDT   
 date                                                                  
 2018-04-17 04:00:00  0.25551    0.288  0.25551  0.26664  8143693.23   
 2018-04-17 05:00:00   0.2666  0.27798   0.2601    0.262  8317923.61   
 2018-04-17 06:00:00  0.26221  0.26396    0.248  0.25664  8420095.41   
 2018-04-17 07:00:00  0.25662    0.263  0.25489  0.25698  4686043.91   
 2018-04-17 08:00:00  0.25636  0.25998  0.25229  0.25631  3510038.13   
 ...                      ...      ...      ...      ...         ...   
 2024-12-27 19:00:00   0.8916   0.8942   0.8874   0.8929   3637923.9   
 2024-12-27 20:00:00    0.893   0.8947   0.8845   0.8887   2724519.1   
 2024-12-27 21:00:00   0.8888   0.8899   0.8757   0.8759   2788857.6   
 2024-12-27 22:00:00    0.876   0.8855   0.8728   0.8744   3688733.3   
 2024-12-27 23:00:00   0.8743   0.8775   0.8699   0.8771   28549

In [17]:
# Get the union of all indices (dates) to align the data
import pandas as pd
from functools import reduce

# Get the union of all indices (dates) to align the data
all_dates = reduce(pd.Index.union, [df.index.get_level_values(0) for df in dfs])

# Reindex all DataFrames to the same set of dates (adding NaNs where data is missing)
dfs_aligned = [df.reindex(all_dates, level=0, fill_value=None) for df in dfs]

# Concatenate all DataFrames
concatenated_df = pd.concat(dfs_aligned, axis=1)
concatenated_df

,open,high,low,close,volume,volume_in_dollars,open,high,low,close,...,low,close,volume,volume_in_dollars,open,high,low,close,volume,volume_in_dollars
coin,ADAUSDT,ADAUSDT,ADAUSDT,ADAUSDT,ADAUSDT,ADAUSDT,ALGOUSDT,ALGOUSDT,ALGOUSDT,ALGOUSDT,...,XMRUSDT,XMRUSDT,XMRUSDT,XMRUSDT,XTZUSDT,XTZUSDT,XTZUSDT,XTZUSDT,XTZUSDT,XTZUSDT
date,,,,,,,,,,,,,,,,,,,,,
2017-08-17 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27 19:00:00,0.8916,0.8942,0.8874,0.8929,3637923.9,3239242.03156,0.3404,0.3411,0.3369,0.3395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-27 20:00:00,0.893,0.8947,0.8845,0.8887,2724519.1,2419769.80538,0.3395,0.3404,0.3354,0.3374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
concatenated_df = concatenated_df.sort_index(axis=1)
concatenated_df

close                                           \
coin                ADAUSDT ALGOUSDT ARUSDT AVAXUSDT BCHUSDT BNBUSDT   
date                                                                   
2017-08-17 04:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 05:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 06:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 07:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
2017-08-17 08:00:00     NaN      NaN    NaN      NaN     NaN     NaN   
...                     ...      ...    ...      ...     ...     ...   
2024-12-27 19:00:00  0.8929   0.3395  16.98    37.19   447.1     NaN   
2024-12-27 20:00:00  0.8887   0.3374  16.97    36.99   445.0     NaN   
2024-12-27 21:00:00  0.8759   0.3324  16.73    36.63   440.8     NaN   
2024-12-27 22:00:00  0.8744   0.3325  16.73    36.45   440.9     NaN   
2024-12-27 23:00:00  0.8771   0.3331  16.74    36.56   441.3     NaN   

                                                        ... volume_in_dollars  \
coin                  BTCUSDT CELOUSDT CHRUSDT CHZUSDT  ...           SKLUSDT   
date                                                    ...                     
2017-08-17 04:00:00   4308.83      NaN     NaN     NaN  ...               NaN   
2017-08-17 05:00:00   4315.32      NaN     NaN     NaN  ...               NaN   
2017-08-17 06:00:00   4324.35      NaN     NaN     NaN  ...               NaN   
2017-08-17 07:00:00   4349.99      NaN     NaN     NaN  ...               NaN   
2017-08-17 08:00:00   4360.69      NaN     NaN     NaN  ...               NaN   
...                       ...      ...     ...     ...  ...               ...   
2024-12-27 19:00:00   94750.0   0.6782  0.2394     NaN  ...      147597.57314   
2024-12-27 20:00:00  94588.12   0.6756   0.238     NaN  ...       92097.98299   
2024-12-27 21:00:00  94526.07   0.6661  0.2353     NaN  ...       93698.29035   
2024-12-27 22:00:00  94437.75   0.6668  0.2349     NaN  ...       54746.25489   
2024-12-27 23:00:00  94299.03   0.6697  0.2359     NaN  ...      106334.05459   

                                                                          \
coin                        SOLUSDT     SYSUSDT THETAUSDT        TRXUSDT   
date                                                                       
2017-08-17 04:00:00             NaN         NaN       NaN            NaN   
2017-08-17 05:00:00             NaN         NaN       NaN            NaN   
2017-08-17 06:00:00             NaN         NaN       NaN            NaN   
2017-08-17 07:00:00             NaN         NaN       NaN            NaN   
2017-08-17 08:00:00             NaN         NaN       NaN            NaN   
...                             ...         ...       ...            ...   
2024-12-27 19:00:00   6679103.77139    62881.21       NaN  5721538.54161   
2024-12-27 20:00:00   8412081.50078  27205.6398       NaN  4796173.85633   
2024-12-27 21:00:00   6719660.39419  37773.5518       NaN  4429262.00457   
2024-12-27 22:00:00  10751316.32927  54086.3708       NaN  7739229.38011   
2024-12-27 23:00:00  14597166.41066  81340.7657       NaN  3885174.56787   

                                                                         \
coin                       VETUSDT     WAXPUSDT         XECUSDT XMRUSDT   
date                                                                      
2017-08-17 04:00:00            NaN          NaN             NaN     NaN   
2017-08-17 05:00:00            NaN          NaN             NaN     NaN   
2017-08-17 06:00:00            NaN          NaN             NaN     NaN   
2017-08-17 07:00:00            NaN          NaN             NaN     NaN   
2017-08-17 08:00:00            NaN          NaN             NaN     NaN   
...                            ...          ...             ...     ...   
2024-12-27 19:00:00  881844.023617  30357.62666  28784.19932494     NaN   
2024-12-27 20:00:00  524184.977356  50791.34977  32587.28112322

In [19]:
concatenated_df = concatenated_df.apply(pd.to_numeric, errors='coerce', downcast='float')

In [20]:
concatenated_df

close                                             \
coin                ADAUSDT ALGOUSDT     ARUSDT   AVAXUSDT     BCHUSDT   
date                                                                     
2017-08-17 04:00:00     NaN      NaN        NaN        NaN         NaN   
2017-08-17 05:00:00     NaN      NaN        NaN        NaN         NaN   
2017-08-17 06:00:00     NaN      NaN        NaN        NaN         NaN   
2017-08-17 07:00:00     NaN      NaN        NaN        NaN         NaN   
2017-08-17 08:00:00     NaN      NaN        NaN        NaN         NaN   
...                     ...      ...        ...        ...         ...   
2024-12-27 19:00:00  0.8929   0.3395  16.980000  37.189999  447.100006   
2024-12-27 20:00:00  0.8887   0.3374  16.969999  36.990002  445.000000   
2024-12-27 21:00:00  0.8759   0.3324  16.730000  36.630001  440.799988   
2024-12-27 22:00:00  0.8744   0.3325  16.730000  36.450001  440.899994   
2024-12-27 23:00:00  0.8771   0.3331  16.740000  36.560001  441.299988   

                                                                ...  \
coin                BNBUSDT   BTCUSDT CELOUSDT CHRUSDT CHZUSDT  ...   
date                                                            ...   
2017-08-17 04:00:00     NaN   4308.83      NaN     NaN     NaN  ...   
2017-08-17 05:00:00     NaN   4315.32      NaN     NaN     NaN  ...   
2017-08-17 06:00:00     NaN   4324.35      NaN     NaN     NaN  ...   
2017-08-17 07:00:00     NaN   4349.99      NaN     NaN     NaN  ...   
2017-08-17 08:00:00     NaN   4360.69      NaN     NaN     NaN  ...   
...                     ...       ...      ...     ...     ...  ...   
2024-12-27 19:00:00     NaN  94750.00   0.6782  0.2394     NaN  ...   
2024-12-27 20:00:00     NaN  94588.12   0.6756  0.2380     NaN  ...   
2024-12-27 21:00:00     NaN  94526.07   0.6661  0.2353     NaN  ...   
2024-12-27 22:00:00     NaN  94437.75   0.6668  0.2349     NaN  ...   
2024-12-27 23:00:00     NaN  94299.03   0.6697  0.2359     NaN  ...   

                    volume_in_dollars                                      \
coin                          SKLUSDT       SOLUSDT     SYSUSDT THETAUSDT   
date                                                                        
2017-08-17 04:00:00               NaN           NaN         NaN       NaN   
2017-08-17 05:00:00               NaN           NaN         NaN       NaN   
2017-08-17 06:00:00               NaN           NaN         NaN       NaN   
2017-08-17 07:00:00               NaN           NaN         NaN       NaN   
2017-08-17 08:00:00               NaN           NaN         NaN       NaN   
...                               ...           ...         ...       ...   
2024-12-27 19:00:00      147597.57314  6.679104e+06  62881.2100       NaN   
2024-12-27 20:00:00       92097.98299  8.412082e+06  27205.6398       NaN   
2024-12-27 21:00:00       93698.29035  6.719660e+06  37773.5518       NaN   
2024-12-27 22:00:00       54746.25489  1.075132e+07  54086.3708       NaN   
2024-12-27 23:00:00      106334.05459  1.459717e+07  81340.7657       NaN   

                                                                             \
coin                      TRXUSDT        VETUSDT     WAXPUSDT       XECUSDT   
date                                                                          
2017-08-17 04:00:00           NaN            NaN          NaN           NaN   
2017-08-17 05:00:00           NaN            NaN          NaN           NaN   
2017-08-17 06:00:00           NaN            NaN          NaN           NaN   
2017-08-17 07:00:00           NaN            NaN          NaN           NaN   
2017-08-17 08:00:00           NaN            NaN          NaN           NaN   
...                           ...            ...          ...           ...   
2024-12-27 19:00:00  5.721539e+06  881844.023617  30357.62666  28784.199325   
2024-12-27 20:00:00  4.796174e+06  524184.977356  50791.34977  32587.281123   
2024-12-27 21:00:00  4.429262e+06  510291.

In [21]:
concatenated_df['close', 'BTCUSDT'].pct_change()

C:\Users\yassi\AppData\Local\Temp\ipykernel_22248\2850305417.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  concatenated_df['close', 'BTCUSDT'].pct_change()


date
2017-08-17 04:00:00         NaN
2017-08-17 05:00:00    0.001506
2017-08-17 06:00:00    0.002093
2017-08-17 07:00:00    0.005929
2017-08-17 08:00:00    0.002460
                         ...   
2024-12-27 19:00:00    0.001945
2024-12-27 20:00:00   -0.001708
2024-12-27 21:00:00   -0.000656
2024-12-27 22:00:00   -0.000934
2024-12-27 23:00:00   -0.001469
Name: (close, BTCUSDT), Length: 64510, dtype: float64

In [22]:
from fetch_symbols import get_symbols
import numpy as np

class CSV_Data:
    def __init__(self, folder_path, symbols):
        self.folder_path = folder_path
        self.symbols = symbols
        self.df = self.process_folder(folder_path, symbols)
        self.df = self.prepare_data()
        self.upload_data_to_csv(self.df)
        
    
    def prepare_data(self):
        df = self.df.copy()
        for coin in df.columns.levels[1]:
            df['returns', coin] = df['close', coin].pct_change()
            df['log_return', coin] = np.log(df['returns', coin])
            df["creturns", coin] = df["log_return", coin].cumsum().apply(np.exp)
            df['price', coin] = df['close', coin]
            df['volume_in_dollars', coin] = df['close', coin] * df['volume', coin]

        df = df.stack(level=1, future_stack=True)
        df.sort_index(axis=1, inplace=True)
        df.index.names = ['date', 'coin']
        df.dropna(inplace=True)

        return df
    
    def get_data(self, file_path, symbols):
        df = pd.read_csv(file_path)
        df = df.drop(columns = df.columns[-1]).reset_index()
        df.drop(columns = df.columns[0], inplace = True)
        df.drop(index = 0, inplace = True)
        df.columns = ['date', 'coin', 'open', 'high', 'low', 'close', 'volume', 'volume_in_dollars']

        if not df['coin'].iloc[0] in symbols:
            return
        # Clean the date column by stripping whitespace
        df['date'] = df['date'].str.strip()
        # Parse the date column with mixed format
        df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
        
        df.set_index([df.columns[0], df.columns[1]], inplace = True)
        df = df.unstack()
        return df
    
    def process_folder(self, folder_path, symbols):
        # Get all CSV files in the folder
        csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
        
        dfs = []
        
        for file in csv_files:
            file_path = os.path.join(folder_path, file)
            df = self.get_data(file_path, symbols)
            if df is not None:
                dfs.append(df)
        

        # Get the union of all indices (dates) to align the data
        all_dates = reduce(pd.Index.union, [df.index.get_level_values(0) for df in dfs])

        # Reindex all DataFrames to the same set of dates (adding NaNs where data is missing)
        dfs_aligned = [df.reindex(all_dates, level=0, fill_value=None) for df in dfs]

        # Concatenate all DataFrames
        concatenated_df = pd.concat(dfs_aligned, axis=1)
        concatenated_df = concatenated_df.sort_index(axis=1)
        concatenated_df = concatenated_df.apply(pd.to_numeric, errors='coerce', downcast='float') #Essential to perform calculations
        
        return concatenated_df

    def upload_data_to_csv(self, df):
        # Upload the data to CSV file
        df.to_csv('all_data.csv')
    

    



# Example usage
# symbols = ['BTCUSDT', 'ETHUSDT']
# interval = '1h'
# start_time = dt.datetime(2020, 1, 1)
# end_time = dt.datetime(2020, 3, 1)
# df = Data(symbols, interval, start_time, end_time).df
# print(df)



# symbols = get_symbols()
# binance_symbols = Data(symbols)
folder_path = r'C:\Users\yassi\OneDrive\Documents\Trading\Algo Trading Projects\Algo Business\data\Binance Data (CSV)'
df = CSV_Data(folder_path, symbols).df

Skipping category 'layer-1', already processed.
Skipping category 'depin', already processed.
Skipping category 'proof-of-work-pow', already processed.
Skipping category 'proof-of-stake-pos', already processed.
Skipping category 'meme-token', already processed.
Skipping category 'dog-themed-coins', already processed.
Skipping category 'eth-2-0-staking', already processed.
Skipping category 'non-fungible-tokens-nft', already processed.
Skipping category 'governance', already processed.
Skipping category 'artificial-intelligence', already processed.
Skipping category 'infrastructure', already processed.
Skipping category 'layer-2', already processed.
Skipping category 'zero-knowledge-zk', already processed.
Skipping category 'storage', already processed.
Skipping category 'oracle', already processed.
Skipping category 'bitcoin-fork', already processed.
Skipping category 'restaking', already processed.
Skipping category 'rollup', already processed.
Skipping category 'metaverse', already p

C:\Users\yassi\AppData\Local\Temp\ipykernel_22248\459813260.py:30: DtypeWarning: Columns (0,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\yassi\AppData\Local\Temp\ipykernel_22248\459813260.py:16: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['returns', coin] = df['close', coin].pct_change()
C:\Users\yassi\AppData\Roaming\Python\Python312\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\yassi\AppData\Roaming\Python\Python312\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\yassi\AppData\Local\Temp\i

In [23]:
df

close      creturns         high  \
date                coin                                               
2017-08-17 05:00:00 BTCUSDT   4315.320000  1.506209e-03  4328.690000   
                    ETHUSDT    303.100006  4.940270e-03   303.279999   
2017-08-17 06:00:00 BTCUSDT   4324.350000  3.151810e-06  4345.450000   
2017-08-17 07:00:00 BTCUSDT   4349.990000  1.868776e-08  4349.990000   
                    ETHUSDT    307.959991  8.617874e-05   307.959991   
...                                   ...           ...          ...   
2024-12-27 23:00:00 SYSUSDT      0.112200  0.000000e+00     0.112400   
                    TRXUSDT      0.259400  0.000000e+00     0.259600   
                    VETUSDT      0.045760  0.000000e+00     0.045760   
                    WAXPUSDT     0.040910  0.000000e+00     0.040940   
                    XECUSDT      0.000035  0.000000e+00     0.000035   

                              log_return          low         open  \
date                coin                                             
2017-08-17 05:00:00 BTCUSDT    -6.498159  4291.370000  4308.830000   
                    ETHUSDT    -5.310335   300.000000   301.609985   
2017-08-17 06:00:00 BTCUSDT    -6.169374  4309.370000  4330.290000   
2017-08-17 07:00:00 BTCUSDT    -5.127863  4287.410000  4316.620000   
                    ETHUSDT    -4.048752   302.600006   302.679993   
...                                  ...          ...          ...   
2024-12-27 23:00:00 SYSUSDT    -5.632999     0.111100     0.111600   
                    TRXUSDT    -7.860292     0.258600     0.259400   
                    VETUSDT    -5.479996     0.045370     0.045580   
                    WAXPUSDT   -5.915972     0.040590     0.040770   
                    XECUSDT    -5.503282     0.000034     0.000034   

                                    price   returns        volume  \
date                coin                                            
2017-08-17 05:00:00 BTCUSDT   4315.320000  0.001506  2.323492e+01   
                    ETHUSDT    303.100006  0.004940  3.776725e+02   
2017-08-17 06:00:00 BTCUSDT   4324.350000  0.002093  7.229691e+00   
2017-08-17 07:00:00 BTCUSDT   4349.990000  0.005929  4.443249e+00   
                    ETHUSDT    307.959991  0.017444  7.547451e+02   
...                                   ...       ...           ...   
2024-12-27 23:00:00 SYSUSDT      0.112200  0.003578  7.290790e+05   
                    TRXUSDT      0.259400  0.000386  1.499596e+07   
                    VETUSDT      0.045760  0.004169  1.004372e+07   
                    WAXPUSDT     0.040910  0.002696  1.056349e+06   
                    XECUSDT      0.000035  0.004073  1.147830e+09   

                              volume_in_dollars  
date                coin                         
2017-08-17 05:00:00 BTCUSDT        1.002661e+05  
                    ETHUSDT        1.144725e+05  
2017-08-17 06:00:00 BTCUSDT        3.126371e+04  
2017-08-17 07:00:00 BTCUSDT        1.932809e+04  
                    ETHUSDT        2.324313e+05  
...                                         ...  
2024-12-27 23:00:00 SYSUSDT        8.180266e+04  
                    TRXUSDT        3.889953e+06  
                    VETUSDT        4.596008e+05  
                    WAXPUSDT       4.321524e+04  
                    XECUSDT        3.961160e+04  

[956964 rows x 10 columns]

In [27]:
pd.read_csv('all_data.csv', index_col=['date', 'coin'], parse_dates=True, low_memory=False)


C:\Users\yassi\AppData\Local\Temp\ipykernel_22248\2691297523.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.read_csv('all_data.csv', index_col=['date', 'coin'], parse_dates=True, low_memory=False)


close      creturns         high  \
date                coin                                               
2017-08-17 05:00:00 BTCUSDT   4315.320000  1.506209e-03  4328.690000   
                    ETHUSDT    303.100006  4.940270e-03   303.279999   
2017-08-17 06:00:00 BTCUSDT   4324.350000  3.151810e-06  4345.450000   
2017-08-17 07:00:00 BTCUSDT   4349.990000  1.868776e-08  4349.990000   
                    ETHUSDT    307.959991  8.617874e-05   307.959991   
...                                   ...           ...          ...   
2024-12-27 23:00:00 SYSUSDT      0.112200  0.000000e+00     0.112400   
                    TRXUSDT      0.259400  0.000000e+00     0.259600   
                    VETUSDT      0.045760  0.000000e+00     0.045760   
                    WAXPUSDT     0.040910  0.000000e+00     0.040940   
                    XECUSDT      0.000035  0.000000e+00     0.000035   

                              log_return          low         open  \
date                coin                                             
2017-08-17 05:00:00 BTCUSDT    -6.498159  4291.370000  4308.830000   
                    ETHUSDT    -5.310335   300.000000   301.609985   
2017-08-17 06:00:00 BTCUSDT    -6.169374  4309.370000  4330.290000   
2017-08-17 07:00:00 BTCUSDT    -5.127863  4287.410000  4316.620000   
                    ETHUSDT    -4.048752   302.600006   302.679993   
...                                  ...          ...          ...   
2024-12-27 23:00:00 SYSUSDT    -5.632999     0.111100     0.111600   
                    TRXUSDT    -7.860292     0.258600     0.259400   
                    VETUSDT    -5.479996     0.045370     0.045580   
                    WAXPUSDT   -5.915972     0.040590     0.040770   
                    XECUSDT    -5.503282     0.000034     0.000034   

                                    price   returns        volume  \
date                coin                                            
2017-08-17 05:00:00 BTCUSDT   4315.320000  0.001506  2.323492e+01   
                    ETHUSDT    303.100006  0.004940  3.776725e+02   
2017-08-17 06:00:00 BTCUSDT   4324.350000  0.002093  7.229691e+00   
2017-08-17 07:00:00 BTCUSDT   4349.990000  0.005929  4.443249e+00   
                    ETHUSDT    307.959991  0.017444  7.547451e+02   
...                                   ...       ...           ...   
2024-12-27 23:00:00 SYSUSDT      0.112200  0.003578  7.290790e+05   
                    TRXUSDT      0.259400  0.000386  1.499596e+07   
                    VETUSDT      0.045760  0.004169  1.004372e+07   
                    WAXPUSDT     0.040910  0.002696  1.056349e+06   
                    XECUSDT      0.000035  0.004073  1.147830e+09   

                              volume_in_dollars  
date                coin                         
2017-08-17 05:00:00 BTCUSDT        1.002661e+05  
                    ETHUSDT        1.144725e+05  
2017-08-17 06:00:00 BTCUSDT        3.126371e+04  
2017-08-17 07:00:00 BTCUSDT        1.932809e+04  
                    ETHUSDT        2.324313e+05  
...                                         ...  
2024-12-27 23:00:00 SYSUSDT        8.180266e+04  
                    TRXUSDT        3.889953e+06  
                    VETUSDT        4.596008e+05  
                    WAXPUSDT       4.321524e+04  
                    XECUSDT        3.961160e+04  

[956964 rows x 10 columns]